In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import pandas as po

## Best Score
Pearson = 0.8621
mem_dim = 100
hidden_dim = 200
lr = 0.03
wd = 0.001
epochs = 10
seed = 1947

In [6]:
!python treelstm/main.py --lr 0.03 --wd 0.001 --optim adagrad --batchsize 25 --seed 1947 --epochs 10 --hidden_dim 200 --mem_dim 100

[2019-10-19 17:06:05,693] DEBUG:__main__:Namespace(batchsize=25, cuda=False, data='data/sick/', epochs=10, evaluate=False, expname='test', freeze_embed=True, glove='data/glove/', hidden_dim=200, input_dim=300, load_model=False, lr=0.03, mem_dim=100, model_type='mdep', num_classes=5, optim='adagrad', save='checkpoints/', saved_model=None, seed=1947, sparse=False, wd=0.001)
[2019-10-19 17:06:05,696] DEBUG:__main__:==> SICK vocabulary size : 2412 
[2019-10-19 17:06:05,697] DEBUG:__main__:==> RELATION vocabulary size : 37 
[2019-10-19 17:06:06,123] DEBUG:__main__:==> Size of train data   : 4500 
[2019-10-19 17:06:06,161] DEBUG:__main__:==> Size of dev data     : 500 
[2019-10-19 17:06:07,011] DEBUG:__main__:==> Size of test data     : 4927 
Training epoch 1:   0%|                                | 0/4500 [00:00<?, ?it/s]/Users/VarunMadhavan/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1946: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the 

Testing epoch  9: 100%|████████████████████| 4927/4927 [00:27<00:00, 181.21it/s]
[2019-10-19 17:39:59,660] INFO:__main__:==> Epoch 8, Train 	Loss: 0.03498136115910165	Pearson: 0.9748976826667786	 spearman:0.9641626358914374	 MSE: 0.05246793478727341
[2019-10-19 17:39:59,661] INFO:__main__:==> Epoch 8, Dev 	Loss: 0.10262515760521637	Pearson: 0.8684377074241638	 spearman:0.8202945068939675	 MSE: 0.24566902220249176
[2019-10-19 17:39:59,663] INFO:__main__:==> Epoch 8, Test 	Loss: 0.10389414016395941	Pearson: 0.8632368445396423	 spearman:0.8064238305114609	 MSE: 0.2614564597606659
[2019-10-19 17:39:59,664] DEBUG:__main__:==> New optimum found, checkpointing everything now...
Testing epoch  10: 100%|███████████████████| 4927/4927 [00:26<00:00, 189.24it/s]
[2019-10-19 17:42:38,005] INFO:__main__:==> Epoch 9, Train 	Loss: 0.03208584707379163	Pearson: 0.9786712527275085	 spearman:0.968783946505918	 MSE: 0.04783012717962265
[2019-10-19 17:42:38,011] INFO:__main__:==> Epoch 9, Dev 	Loss: 0.10449

In [119]:
results = po.DataFrame(columns = ['hidden_dim', 'mem_dim', 'lr', 'pearson', 'spearman', 'mse'])
results.to_csv('checkpoints/mdep/Results.csv', index = False)

In [126]:
!python treelstm/main.py --lr 0.05 --wd 0.0001 --opti m adagrad --batchsize 25 --freeze_embed --epochs 1 --hidden_dim 60 --mem_dim 100 

[2019-10-13 15:20:10,906] DEBUG:__main__:Namespace(batchsize=25, cuda=False, data='data/sick/', epochs=1, evaluate=False, expname='test', freeze_embed=False, glove='data/glove/', hidden_dim=60, input_dim=300, load_model=False, lr=0.05, mem_dim=100, model_type='mdep', num_classes=5, optim='adagrad', save='checkpoints/', saved_model=None, seed=123, sparse=False, wd=0.0001)
[2019-10-13 15:20:10,909] DEBUG:__main__:==> SICK vocabulary size : 2412 
[2019-10-13 15:20:10,909] DEBUG:__main__:==> RELATION vocabulary size : 37 
[2019-10-13 15:20:11,390] DEBUG:__main__:==> Size of train data   : 4500 
[2019-10-13 15:20:11,424] DEBUG:__main__:==> Size of dev data     : 500 
[2019-10-13 15:20:12,168] DEBUG:__main__:==> Size of test data     : 4927 
Training epoch 1:   0%|                                | 0/4500 [00:00<?, ?it/s]/Users/VarunMadhavan/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1946: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the s

In [114]:
dict = {'hidden_dim': 50,
        'lr': 0.05,
        'pearson': 0.5673,
        'spearman': 0.5711,
        'mse': 0.6944,
        'mem_dim': 150
       }

results = results.append(dict, ignore_index=True)
results.to_csv('checkpoints/mdep/Results.csv', index = False)

In [121]:
results=po.read_csv('checkpoints/mdep/Results.csv')
results

,hidden_dim,mem_dim,lr,pearson,spearman,mse
0,50,150,0.05,0.5712,0.5642,0.7282


In [122]:
mse = results.iloc[-1]['mse']
mse

0.7282

In [125]:
from hyperopt import fmin,tpe,hp, STATUS_OK
best_mse = 1000
def f(space):
    
    global best_h_d, best_m_d, best_lr, best_mse
    hidden_dim = space['hidden_dim']
    lr = space['lr']
    mem_dim = space['mem_dim']
    
    !python treelstm/main.py --wd 0.0001 --optim adagrad --batchsize 25 --freeze_embed --epochs 1 --hidden_dim hidden_dim --mem_dim mem_dim 
    results=po.read_csv('checkpoints/mdep/Results.csv')
    mse = results.iloc[-1]['mse']
    
    if (mse < best_mse):
        best_m_d = m_d
        best_h_d = h_d
        best_lr = lr
        best_mse = mse
        print('best_mse = {}'.format(best_mse))
        print('best_h_d = {}'.format(h_d))
        print('best_lr = {}'.format(lr))
        print('best_m_d = {}'.format(m_d))
    
    return {'loss': mse, 'status': STATUS_OK }

space = {
    'hidden_dim': hp.quniform('hidden_dim', 10, 200,10),
    'mem_dim': hp.quniform('mem_dim',10,200,10),
    'lr': hp.quniform('lr',0.01,1,0.01),
    }

best = fmin(fn=f,space=space,algo=tpe.suggest,max_evals=150)

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--lr LR] [--wd WD] [--sparse]
               [--optim OPTIM] [--seed SEED] [--cuda | --no-cuda]
main.py: error: argument --hidden_dim: invalid int value: 'h_d'

best_mse = 0.7282                                    
best_h_d = 200.0                                     
best_lr = 0.48                                       
best_m_d = 150.0                                     
usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]   
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidd

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]    
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--lr LR] [--wd WD] [--sparse]
               [--optim OPTIM] [--seed SEED] [--cuda | --no-cuda]
main.py: error: argument --hidden_dim: invalid int value: 'h_d'

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]    
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--l

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]    
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--lr LR] [--wd WD] [--sparse]
               [--optim OPTIM] [--seed SEED] [--cuda | --no-cuda]
main.py: error: argument --hidden_dim: invalid int value: 'h_d'

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]    
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--l

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]    
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--lr LR] [--wd WD] [--sparse]
               [--optim OPTIM] [--seed SEED] [--cuda | --no-cuda]
main.py: error: argument --hidden_dim: invalid int value: 'h_d'

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]    
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--l

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]    
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--lr LR] [--wd WD] [--sparse]
               [--optim OPTIM] [--seed SEED] [--cuda | --no-cuda]
main.py: error: argument --hidden_dim: invalid int value: 'h_d'

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]    
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--l

...

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]    
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--lr LR] [--wd WD] [--sparse]
               [--optim OPTIM] [--seed SEED] [--cuda | --no-cuda]
main.py: error: argument --hidden_dim: invalid int value: 'h_d'

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]    
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--l

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]     
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--lr LR] [--wd WD] [--sparse]
               [--optim OPTIM] [--seed SEED] [--cuda | --no-cuda]
main.py: error: argument --hidden_dim: invalid int value: 'h_d'

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]     
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [-

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]     
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--lr LR] [--wd WD] [--sparse]
               [--optim OPTIM] [--seed SEED] [--cuda | --no-cuda]
main.py: error: argument --hidden_dim: invalid int value: 'h_d'

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]     
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [-

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]     
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--lr LR] [--wd WD] [--sparse]
               [--optim OPTIM] [--seed SEED] [--cuda | --no-cuda]
main.py: error: argument --hidden_dim: invalid int value: 'h_d'

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]     
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [-

usage: main.py [-h] [--data DATA] [--glove GLOVE] [--save SAVE]     
               [--expname EXPNAME] [--model_type MODEL_TYPE]
               [--input_dim INPUT_DIM] [--mem_dim MEM_DIM]
               [--hidden_dim HIDDEN_DIM] [--num_classes NUM_CLASSES]
               [--freeze_embed] [--load_model] [--evaluate]
               [--saved_model SAVED_MODEL] [--epochs EPOCHS]
               [--batchsize BATCHSIZE] [--lr LR] [--wd WD] [--sparse]
               [--optim OPTIM] [--seed SEED] [--cuda | --no-cuda]
main.py: error: argument --hidden_dim: invalid int value: 'h_d'

100%|██████████| 150/150 [02:29<00:00,  1.00it/s, best loss: 0.7282]


In [79]:
space['hidden_dim']